## ДЗ по поиску

Привет! Вам надо реализивать поисковик на базе вопросов-ответов с сайта [pravoved.ru](https://pravoved.ru/questions-archive/).        
Поиск должен работать на трех технологиях:       
1. обратном индексе     
2. word2vec         
3. doc2vec      

Вы должны понять, какой метод и при каких условиях эксперимента на этом корпусе работает лучше.          
Для измерения качества поиска найдите точность (accuracy) выпадания правильного ответа на конкретный вопрос (в этой базе у каждого вопроса есть только один правильный ответ). Точность нужно измерить для всей базы.    
При этом давайте считать, что выпал правильный ответ, если он попал в **топ-5** поисковой выдачи.

> Сделайте ваш поиск максимально качественным, чтобы значение точности стремилось к 1.     
Для этого можно поэкспериментировать со следующим:       
- модель word2vec (можно брать любую из опен сорса или обучить свою)
- способ получения вектора документа через word2vec: простое среднее арифметическое или взвешивать каждый вектор в соответствии с его tf-idf      
- количество эпох у doc2vec (начинайте от 100)
- предобработка документов для обучения doc2vec (удалять / не удалять стоп-слова)
- блендинг методов поиска: соединить результаты обратного индекса и w2v, или (что проще) w2v и d2v

На это задание отведем 10 дней. Дэдлайн сдачи до полуночи 12.10.

In [1]:
import pickle

with open('qa_corpus.pkl', 'rb') as file:
    qa_corpus = pickle.load(file)

Всего в корпусе 1384 пары вопрос-ответ

In [2]:
len(qa_corpus)

1384

Первый элемент блока это вопрос, второй - ответ на него

In [15]:
qa_corpus[0][0]

'\nДобрый день.Мой сын гражданин Украины (ДНР),имеет вид на жительство в Р.Ф., кот.получил проживая с 2014 г. в Нижегородской области.В 2017г. переехал на постоянное место жительство в г.Ростов.Официально трудоустроился на одно из промышл.предприятий г.Ростова.Оформил временную регистрацию в Ростове.В УФМС предупредили,что по истечении 90 дней он должен либо постоянно прописаться либо покинуть территорию России.Прошу проконсультировать как быть дальше.(Вернуться домой в Донецк,но здесь идет война,работы нет.В Ростове он работает по специальности.Он инженер машиностроитель.)Временная прописка до 15 марта.  Если он сможет приобрести какую либо недвижимость,как долго будет решаться вопрос о его постоянной прописке в Ростове.Как в этом случае будет решаться вопрос с видом на жительство в Ростове? Не получится ли ,что приобретя квартиру,он не успеет в ней прописаться до окончании срока временной регистрации. С уважением Людмила Евгеньевна.\n'

In [4]:
from nltk.corpus import stopwords

In [5]:
import nltk
from nltk.tokenize import word_tokenize

In [6]:
from pymystem3 import Mystem
mystem = Mystem()

In [7]:
from string import punctuation

In [8]:
def preprocessing(input_text, del_stopwords=True, del_digit=True):
    """
    :input: raw text
    1. lowercase, del punctuation, tokenize
    2. normal form
    3. del stopwords
    4. del digits
    :return: lemmas
    """
    russian_stopwords = set(stopwords.words('russian'))
    words = [x.lower().strip(punctuation+'»«–…') for x in word_tokenize(input_text)]
    lemmas = [mystem.lemmatize(x)[0] for x in words if x]

    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
    return lemmas_arr

In [19]:
def get_information(arr):
    """
    Create inverted index by input doc collection
    Get essential information from input doc collection
    :return: inverted index, 
    """
    dictionary = {}
    question_len = {}
    for block in arr:
        words = preprocessing(block[0])
        question_len[arr.index(block)] = len(words)
        counter = Counter(words)
        for word in counter:
            if word in dictionary:
                dictionary[word][arr.index(block)] = counter[word]
            else:
                dictionary[word] = {}
                dictionary[word][arr.index(block)] = counter[word]
    return question_len, dictionary

In [27]:
from collections import Counter

question_len, index = get_information(qa_corpus)

In [23]:
import math
from math import log
import os

In [24]:
k1 = 2.0
b = 0.75

def score_BM25(qf, dl, avgdl, k1, b, N, n) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    score = math.log((N-n+0.5)/(n+0.5)) * (k1+1)*qf/(qf+k1*(1-b+b*(dl/avgdl)))
    return score

In [28]:
def compute_sim(word, index, question_len):
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    N = len(question_len)
    avgdl = sum(question_len.values())/N
    if word in index:
        n = len(index[word])
        result = {}
        for block in index[word]:
            qf = index[word][block]
            score = score_BM25(qf, question_len[block], avgdl, k1, b, N, n)
            result[block] = score
        return result
    else:
        return {}


def get_search_result(inquiry):
    """
    Compute sim score between search query and all documents in collection
    :return: list of files
    """
    global index, question_len
    score = defaultdict(int)
    words = preprocessing(inquiry)
    for word in words:
        result = compute_sim(word, index, question_len)
        for block in result:
            score[block] += result[block]  
        
    return sorted(score, key=score.get, reverse = True)[:5]

In [42]:
from collections import defaultdict

def get_accuracy(arr):
    acc_score = 0
    for index, value in enumerate(arr):
        answers = get_search_result(value[0])
        if index in answers:
            acc_score+=1
    return acc_score/len(arr)

In [43]:
get_accuracy(qa_corpus)

1.0

И правда, инвертированный индекс получился очень эффективный. Рассмотрим несколько примеров:

In [45]:
get_search_result(qa_corpus[10][0])

[10, 555, 1383, 1095, 1072]

In [46]:
get_search_result(qa_corpus[189][0])

[189, 226, 710, 1248, 1227]

In [47]:
get_search_result(qa_corpus[557][0])

[557, 561, 305, 817, 828]

Для того, чтобы поиск выдал ответ на вопрос, остаётся только вернуть вместо индекса сам ответ, который лежит в массиве по этому индексу, а это сделать очень просто.

In [48]:
from gensim.models import Word2Vec, KeyedVectors

In [49]:
w2v_model = Word2Vec.load('/users/kata/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model')

In [50]:
def get_w2v_vectors(question, model): 
    """
    Получает вектор вопроса
    """
    n = 0
    vector = [0] * 300
    lemmas = preprocessing(question)
    for lemma in lemmas:
        try:
            vector += model.wv[lemma]
            n += 1
        except:
            None
    if n != 0:
        vector = vector / n
    return vector

def save_w2v_base(arr, model):
    """Индексирует всю базу для поиска через word2vec"""
    w2v_base = []
    for block in arr:
        vec = {}
        vec['id'] = arr.index(block)
        vec['question'] = block[0]
        vec['answer'] = block[1]
        vec['vector'] = get_w2v_vectors(block[0], model)
        w2v_base.append(vec)

    return w2v_base

In [51]:
w2v_base = save_w2v_base(qa_corpus, w2v_model)

In [53]:
w2v_base[:5]

[{'answer': 'Добрый вечер!Из Вашего вопроса вообще ничего не ясно.Ваш сын по ВНЖ в Нижегородской обл. сделал временную\xa0 на 90 дней в Ростове? Так? Или в чем заключается вопрос?С ув., АлёнаМиграционный юристРостов-на-Дону ',
  'id': 0,
  'question': '\nДобрый день.Мой сын гражданин Украины (ДНР),имеет вид на жительство в Р.Ф., кот.получил проживая с 2014 г. в Нижегородской области.В 2017г. переехал на постоянное место жительство в г.Ростов.Официально трудоустроился на одно из промышл.предприятий г.Ростова.Оформил временную регистрацию в Ростове.В УФМС предупредили,что по истечении 90 дней он должен либо постоянно прописаться либо покинуть территорию России.Прошу проконсультировать как быть дальше.(Вернуться домой в Донецк,но здесь идет война,работы нет.В Ростове он работает по специальности.Он инженер машиностроитель.)Временная прописка до 15 марта.  Если он сможет приобрести какую либо недвижимость,как долго будет решаться вопрос о его постоянной прописке в Ростове.Как в этом случае

In [54]:
from gensim import matutils
import numpy as np 

def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    return np.dot(v1_norm, v2_norm)

In [62]:
def search_w2v(inquiry):
    global w2v_base, w2v_model
    v1 = get_w2v_vectors(inquiry, w2v_model)
    score = defaultdict(int)
    for vec in w2v_base:
        v2 = vec['vector']
        sim = similarity(v1, v2)
        score[vec['id']] = sim
    return sorted(score, key=score.get, reverse = True)[:5]

In [63]:
search_w2v(qa_corpus[0][0])

/Users/kata/anaconda/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[0, 1055, 749, 133, 478]

In [64]:
def get_w2v_accuracy(arr):
    acc_score = 0
    for index, value in enumerate(arr):
        answers = search_w2v(value[0])
        if index in answers:
            acc_score+=1
    return acc_score/len(arr)

In [65]:
get_w2v_accuracy(qa_corpus)

/Users/kata/anaconda/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


1.0

In [70]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


In [67]:
def get_tagged_data(arr):
    tagged_data = []
    i = 0
    for block in arr:
        try:
            data = preprocessing(block[0], del_stopwords=False)
            tagged_data.append(TaggedDocument(words=data, tags=[i]))
            i+=1
        except:
            None
    return tagged_data

In [86]:
def train_doc2vec(tagged_data):
    model = Doc2Vec(vector_size=100, min_count=5, alpha=0.025, min_alpha=0.025, epochs=100, workers=2, dm=1)
    model.build_vocab(tagged_data)
    model.random.seed(12345)
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
    
    return model

In [71]:
d2w_model = train_doc2vec(get_tagged_data(qa_corpus))

In [72]:
def get_d2v_vectors(text, model):
    """Получает вектор документа"""
    return model.infer_vector(text)

def save_d2v_base(arr, model):
    """Индексирует всю базу для поиска через doc2vec"""
    d2v_base = []
    for block in arr:
        vec = {}
        vec['id'] = arr.index(block)
        vec['question'] = block[0]
        vec['answer'] = block[1]
        vec['vector'] = get_d2v_vectors(block[0], model)
        d2v_base.append(vec)
    return d2v_base

In [73]:
d2v_base = save_d2v_base(qa_corpus, d2w_model)

In [75]:
d2v_base[:2]

[{'answer': 'Добрый вечер!Из Вашего вопроса вообще ничего не ясно.Ваш сын по ВНЖ в Нижегородской обл. сделал временную\xa0 на 90 дней в Ростове? Так? Или в чем заключается вопрос?С ув., АлёнаМиграционный юристРостов-на-Дону ',
  'id': 0,
  'question': '\nДобрый день.Мой сын гражданин Украины (ДНР),имеет вид на жительство в Р.Ф., кот.получил проживая с 2014 г. в Нижегородской области.В 2017г. переехал на постоянное место жительство в г.Ростов.Официально трудоустроился на одно из промышл.предприятий г.Ростова.Оформил временную регистрацию в Ростове.В УФМС предупредили,что по истечении 90 дней он должен либо постоянно прописаться либо покинуть территорию России.Прошу проконсультировать как быть дальше.(Вернуться домой в Донецк,но здесь идет война,работы нет.В Ростове он работает по специальности.Он инженер машиностроитель.)Временная прописка до 15 марта.  Если он сможет приобрести какую либо недвижимость,как долго будет решаться вопрос о его постоянной прописке в Ростове.Как в этом случае

In [76]:
def search_d2v(inquiry):
    global d2v_base, d2w_model
    v1 = get_d2v_vectors(inquiry, d2w_model)
    score = defaultdict(int)
    for vec in d2v_base:
        v2 = vec['vector']
        sim = similarity(v1, v2)
        score[vec['id']] = sim
    return sorted(score, key=score.get, reverse = True)[:5]

In [78]:
search_d2v(qa_corpus[1285][0])

/Users/kata/anaconda/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[1285, 1071, 909, 86, 146]

In [79]:
search_d2v(qa_corpus[25][0])

/Users/kata/anaconda/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[25, 236, 256, 1159, 241]

In [80]:
def get_d2v_accuracy(arr):
    acc_score = 0
    for index, value in enumerate(arr):
        answers = search_d2v(value[0])
        if index in answers:
            acc_score+=1
    return acc_score/len(arr)

In [82]:
get_d2v_accuracy(qa_corpus)

/Users/kata/anaconda/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.9819364161849711

Как видно, doc2vec работает не идеально. Попробуем увеличить количество эпох:

In [85]:
def train_doc2vec1(tagged_data):
    model = Doc2Vec(vector_size=100, min_count=5, alpha=0.025, min_alpha=0.025, epochs=200, workers=2, dm=1)
    model.build_vocab(tagged_data)
    model.random.seed(12345)
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
    
    return model

In [87]:
d2w_model1 = train_doc2vec1(get_tagged_data(qa_corpus))

In [89]:
d2v_base1 = save_d2v_base(qa_corpus, d2w_model1)

In [90]:
def search_d2v1(inquiry):
    global d2v_base1, d2w_model1
    v1 = get_d2v_vectors(inquiry, d2w_model)
    score = defaultdict(int)
    for vec in d2v_base:
        v2 = vec['vector']
        sim = similarity(v1, v2)
        score[vec['id']] = sim
    return sorted(score, key=score.get, reverse = True)[:5]

In [92]:
def get_d2v_accuracy1(arr):
    acc_score = 0
    for index, value in enumerate(arr):
        answers = search_d2v1(value[0])
        if index in answers:
            acc_score+=1
    return acc_score/len(arr)

In [93]:
get_d2v_accuracy1(qa_corpus)

/Users/kata/anaconda/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.9747109826589595

Нет, качество только ухудшилось :с Значит, 100 это оптимально.